## Word Embedding training for Fast Text and Doc2Vec vectors

In [4]:
import pandas as pd
indeed65k_90k_ds = pd.read_csv('indeed65k_90k_145k_dssf.csv')

In [5]:
indeed65k_90k_ds_text = indeed65k_90k_ds['summary']

In [6]:
import gensim, logging

In [7]:
bookpossentences=[]
allsentences=[]
with open("indeed65k_90k_ds_text.txt") as f:
    File = f.readlines()
    for l in File:
        words = l.split(" ")
        for i in range(len(words)):
            words[i] = words[i].lower()
            if "," in words[i]:
                words[i]=words[i].replace(",","")
            
            if '\"' in words[i]:
                words[i] = words[i].replace('\"','')
            if "." in words[i]:
                j = words[i].index(".")
                words[i] = words[i][:j]
            #words[-1] = words[-1].replace("\n","")
            elif "\n" in words[i]:
                j = words[i].index("\n")
                words[i] = words[i][:j]
        bookpossentences.append(words)
        allsentences.append(words)
    print words

['ability', 'to', 'face', 'client', 'technical', 'architects', 'and', 'data', 'scientists', 'to', 'explain', 'how', 'we', 'deliver', 'our', 'solutions', 'products', 'and', 'platform']


In [8]:
#from gensim.models.wrappers.fasttext import FastText
from gensim.models.fasttext import FastText
from gensim.models.doc2vec import Doc2Vec

In [9]:
FastTextModel500 =FastText(min_count=1,size=500)
#FastTextModel.build_vocab(allsentences)
FastTextModel500.build_vocab(allsentences)
FastTextModel500.train(allsentences)

In [10]:
def FastVectorizer(text):
    words = text.split()
    for i in range(len(words)):
            words[i] = words[i].lower()
            if "," in words[i]:
                words[i]=words[i].replace(",","")
            
            if '\"' in words[i]:
                words[i] = words[i].replace('\"','')
            if "." in words[i]:
                j = words[i].index(".")
                words[i] = words[i][:j]
            #words[-1] = words[-1].replace("\n","")
            elif "\n" in words[i]:
                j = words[i].index("\n")
                words[i] = words[i][:j]
    s = [0]*500
    for t in range(0,len(words)):
        s = FastTextModel500[words[t]] + s
        #FastTextModel500
    return np.divide(s, len(words))

In [ ]:
import numpy as np
indeed65k_90k_ds['AvgFTVector'] = indeed65k_90k_ds['summary'].apply(lambda x: FastVectorizer(x))

In [ ]:
from gensim.models.doc2vec import TaggedLineDocument
sentences = TaggedLineDocument("indeed65k_90k_ds_text.txt")

In [ ]:
d2vmodel = Doc2Vec(alpha=0.025, min_alpha=0.025)
d2vmodel.build_vocab(sentences)

for epoch in range(10):
    d2vmodel.train(sentences,total_examples=d2vmodel.corpus_count,epochs=d2vmodel.iter)
    d2vmodel.alpha -= 0.002
    d2vmodel.min_alpha = d2vmodel.alpha
    print epoch

In [ ]:
indeed65k_90k_ds['D2Vector'] = indeed65k_90k_ds['summary'].apply(lambda x: d2vmodel.infer_vector(x))

In [ ]:
indeed65k_90k_ds.to_csv('indeed65k_90k_145k_ds_w2v_d2v.txt',index=False)